## Structured Output
Models can be requested to provide their response in a format matching a given schema. This is useful for ensuring the output can be easily parsed and used in subsequent processing. LangChain supports multiple schema types and methods for enforcing structured output.

Two Patterns of Structured Output. They are:

1. Structured output with `PydanticOutputParser`
2. Structured output via tool-calling agents

### Structured output with PydanticOutputParser

In [1]:
from pydantic import BaseModel, Field
from typing import Optional
from typing import List

class TaskPlan(BaseModel):
    task: str = Field(description="The main task to be accomplished")
    steps: List[str] = Field(description="A list of steps to accomplish the task")
    tools_needed: List[str] = Field(description="A list of tools required for the task")
    

In [4]:
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate

parser = PydanticOutputParser(pydantic_object=TaskPlan)

prompt = PromptTemplate(
    template="""
You are a planning agent.

{format_instructions}

User request:
{query}
""",
    input_variables=["query"],
    partial_variables={
        "format_instructions": parser.get_format_instructions()
    },
)

In [ ]:
from langchain_ollama import ChatOllama

OLLAMA_URL = "" # Set your Ollama URL here
OLLAMA_MODEL = "granite4:tiny-h"

model = ChatOllama(model=OLLAMA_MODEL, base_url=OLLAMA_URL)

In [6]:
chain = prompt | model | parser
result = chain.invoke({
    "query": "Build a study plan to learn Python in 30 days"
})

print(result)
print(type(result)) 

task='Build a study plan to learn Python in 30 days' steps=['Set clear goals and objectives for what you want to achieve with Python within 30 days', 'Create a daily schedule allocating dedicated time slots for learning Python each day', 'Gather resources such as online tutorials, books, courses, and coding exercises specific to Python', 'Start with the basics of Python syntax, data types, variables, and control structures', 'Practice writing small programs and solving coding challenges regularly', 'Work on a mini-project that incorporates various concepts learned so far', 'Explore intermediate topics like functions, modules, object-oriented programming, and file handling', 'Engage with online communities or forums to ask questions and learn from others', 'Participate in coding competitions or hackathons to apply your Python skills under time pressure', 'Continuously review and reinforce the concepts learned through quizzes and mock tests', 'Create a portfolio of projects showcasing yo

### Structured output via tool-calling agents

In [20]:
from pydantic import BaseModel, Field

class BioData(BaseModel):
    name: str = Field(description="Name of the person")
    age: int = Field(description="Age of the person")
    occupation: str = Field(description="Occupation of the person")


In [21]:
from langchain_core.tools import tool

@tool
def get_details(name: str, age: int, occupation: str) -> BioData:
    """Get the details of a person."""
    return BioData(
        name=name,
        age=age,
        occupation=occupation
    )


In [22]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You extract personal details and call tools when appropriate."),
    ("human", "{input}")
])


In [23]:
agent = prompt | model
result = agent.invoke({
    "input": "My name is John. I am 28 years old and I work as a software engineer."
})

print(result)


content='Name: John\nAge: 28 \nOccupation: Software Engineer' additional_kwargs={} response_metadata={'model': 'granite4:tiny-h', 'created_at': '2026-01-03T15:37:01.1113092Z', 'done': True, 'done_reason': 'stop', 'total_duration': 423988200, 'load_duration': 39614700, 'prompt_eval_count': 42, 'prompt_eval_duration': 212770900, 'eval_count': 15, 'eval_duration': 161453200, 'logprobs': None, 'model_name': 'granite4:tiny-h', 'model_provider': 'ollama'} id='lc_run--019b8481-358a-7052-9b19-76cddd4d5508-0' tool_calls=[] invalid_tool_calls=[] usage_metadata={'input_tokens': 42, 'output_tokens': 15, 'total_tokens': 57}


## Conclusion

In this notebook, we explored two patterns for structured output:

1. **PydanticOutputParser**: This approach defines a schema using Pydantic models and instructs the model to output data that conforms to that schema. The parser then converts the model's response into a structured Python object.

2. **Tool-calling agents**: This approach uses agents with tools that have structured inputs and outputs. The agent can call these tools to perform specific actions and receive structured responses.

Both approaches are valuable for different use cases:
- Use PydanticOutputParser when you need to extract specific information from unstructured text.
- Use tool-calling agents when you need to perform actions or interact with external systems in a structured way.